# Context | コンテキスト

> [Context](https://context.ai/)は、LLM（Large Language Models）を活用した製品や機能のためのユーザー分析を提供します。
>
> > [Context](https://context.ai/) provides user analytics for LLM-powered products and features.

`Context`を使用することで、30分以内にユーザーを理解し、彼らの体験を向上させることができます。

> With `Context`, you can start understanding your users and improving their experiences in less than 30 minutes.




このガイドでは、Contextとの統合方法をご紹介します。

> In this guide we will show you how to integrate with Context.




## Installation and Setup | インストールとセットアップ




In [ ]:
!pip install context-python --upgrade

### Getting API Credentials | APIクレデンシャルの取得

Context APIトークンを取得するには：

> To get your Context API token:

1. Contextアカウント内の設定ページにアクセスしてください（https://with.context.ai/settings）。

   > Go to the settings page within your Context account (https://with.context.ai/settings).

2. 新しいAPIトークンを生成してください。

   > Generate a new API Token.

3. このトークンを安全な場所に保管してください。

   > Store this token somewhere secure.





### Setup Context | コンテキストの設定

`ContextCallbackHandler`を使用するには、Langchainからハンドラをインポートして、あなたのContext APIトークンを使ってインスタンス化してください。

> To use the `ContextCallbackHandler`, import the handler from Langchain and instantiate it with your Context API token.

ハンドラを使用する前に、`context-python`パッケージがインストールされていることを確認してください。

> Ensure you have installed the `context-python` package before using the handler.




In [3]:
import os

from langchain.callbacks import ContextCallbackHandler

token = os.environ["CONTEXT_API_TOKEN"]

context_callback = ContextCallbackHandler(token)

## Usage | 使用方法

### Context callback within a chat model | チャットモデル内のコンテキストコールバック

Context コールバックハンドラは、ユーザーとAIアシスタント間のトランスクリプトを直接記録するために使用できます。

> The Context callback handler can be used to directly record transcripts between users and AI assistants.




In [4]:
import os

from langchain.callbacks import ContextCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

token = os.environ["CONTEXT_API_TOKEN"]

chat = ChatOpenAI(
    headers={"user_id": "123"}, temperature=0, callbacks=[ContextCallbackHandler(token)]
)

messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(content="I love programming."),
]

print(chat(messages))

### Context callback within Chains | チェーン内のコンテキストコールバック

Context コールバックハンドラは、チェーンの入力と出力を記録するためにも使用できます。チェーンの中間ステップは記録されないことに注意してください - 記録されるのは開始時の入力と最終的な出力のみです。

> The Context callback handler can also be used to record the inputs and outputs of chains. Note that intermediate steps of the chain are not recorded - only the starting inputs and final outputs.

**注意:** チャットモデルとチェーンに同じコンテキストオブジェクトを渡すようにしてください。

> **Note:** Ensure that you pass the same context object to the chat model and the chain.

誤り：

> Wrong:

> ```python
> chat = ChatOpenAI(temperature=0.9, callbacks=[ContextCallbackHandler(token)])
> chain = LLMChain(llm=chat, prompt=chat_prompt_template, callbacks=[ContextCallbackHandler(token)])
> ```

正しい：

> Correct:

> ```python
> handler = ContextCallbackHandler(token)
> chat = ChatOpenAI(temperature=0.9, callbacks=[callback])
> chain = LLMChain(llm=chat, prompt=chat_prompt_template, callbacks=[callback])
> ```




In [ ]:
import os

from langchain.callbacks import ContextCallbackHandler
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

token = os.environ["CONTEXT_API_TOKEN"]

human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="What is a good name for a company that makes {product}?",
        input_variables=["product"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
callback = ContextCallbackHandler(token)
chat = ChatOpenAI(temperature=0.9, callbacks=[callback])
chain = LLMChain(llm=chat, prompt=chat_prompt_template, callbacks=[callback])
print(chain.run("colorful socks"))